In [1]:
import requests
import json
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time
import os

In [2]:
def review_crawling(url):
    url1 = url + "visitor" #리뷰게시판
    url2 = url + "ugc" #블로그리뷰
    
    #1. 리뷰 페이지 이동
    driver.get(url1)
    
    #2. review 긴 내용 모두 클릭
    reviews = driver.find_elements(By.CSS_SELECTOR, 'div.ZZ4OK > a')
    for i in range(len(reviews)):
        driver.execute_script("arguments[0].click();", reviews[i])
    
    #3.페이지 bs4로 받아오기
    req = driver.page_source
    body = BeautifulSoup(req, 'html.parser').body
    
    #4-1. 이름 가져오기
    name = body.select('#_title>span.Fc1rA')[0].text
    
    #4-2.메인 포토 가져오기 (포토가 없을 경우 거리뷰 url 가져오기 )
    try:
        main_photo = body.select_one('#_autoPlayable > div')['style'].split('"')[1] #메인 사진
    except:
        main_photo = body.select_one('div.CB8aP > div > a > div')['style'].split('"')[1] #로드뷰
    
    #4-3.  이런 점이 좋았어요 가져와서 상위 3개만 사용(없을 경우에는 빈칸으로 입력)
    #이런 정ㅁ이 좋았어요 선택한 사람 수 가져오기
    review_summaries = body.select('span.nWiXa')
    review_sum_count = body.select('span.TwM9q')
    if not review_summaries:#없을 경우
        review_summaries = ['']*3
        review_sum_count = ['']*3
    else:#있을 경우
        for i,review_summary in enumerate(review_summaries):
            review_summaries[i]=review_summary.text
            review_sum_count[i]=review_sum_count[i].text.replace('이 키워드를 선택한 인원','')
    
    #4-4. review들,review개수 가져오기
    reviews = body.select('div.ZZ4OK > a > .zPfVt')
    review_count = body.select_one('span.place_section_count')
    if review_count:#리뷰가 있는 경우
        review_count = review_count.text
    else:#리뷰가 없는 경우
        review_count = ''
        
    #4-5.blog리뷰 이동 -> 페이지 bs4로 받아오기 -> 블로그 리뷰 count 받아오기
    driver.get(url2)
    driver.implicitly_wait(5)
    body = BeautifulSoup(driver.page_source, 'html.parser').body
    review_blog_count = body.select_one('em.place_section_count')
    if review_blog_count:#블로그 리뷰가 있는 경우
        review_blog_count = review_blog_count.text
    else:#블로그 리뷰가 없는 경우
        review_blog_count = ''
    review_blog_title = body.select('.place_bluelink.XhMTc')
    review_blog_content = body.select('.j0LvW')
    review_blog_url = body.select('.X_mSv')
    
    data = {
        'url':url,
        'name':name,
        'place_num':place_num,
        'main_jpg':main_photo,
        'review_summary1':review_summaries[0] + ','+review_sum_count[0],
        'review_summary2':review_summaries[1]+ ','+review_sum_count[1],
        'review_summary3':review_summaries[2]+ ','+review_sum_count[2],
        'review_blog_count':review_blog_count,
        'review_count':review_count,
    }
    
    #review
    for i in range(10):
        try:
            data['review'+str(i+1)] = reviews[i].text
        except:#10보다 작은 경우
            data['review'+str(i+1)]=''

    print(len(review_blog_url))           
    for i in range(10):
        try:
            data['review_blog'+str(i+1)] = review_blog_title[i].text +','+review_blog_content[i].text +','+review_blog_url[i].attrs['href']
        except:#10보다 작은 경우
            data['review_blog'+str(i+1)] = ','+ ','
    
    df = pd.DataFrame(data,index=[0])
    if not os.path.exists('review.csv'):
        df.to_csv('review.csv', index=False, mode='w', encoding='utf-8-sig')
    else:
        df.to_csv('review.csv', index=False, mode='a', encoding='utf-8-sig', header=False)
    print(name)

In [ ]:
driver = webdriver.Chrome()
driver.implicitly_wait(5)

# reveiw 페이지 가져오기
detail_url = pd.read_csv('yongsangu.csv')['detail_url']
for i,url in enumerate(detail_url[-40:]):
    print(i,end=' ')
    print(url,end=' ')
    place_num = url.split('/')[7].split('?')[0] # review 페이지 num 가져오기
    url = "https://pcmap.place.naver.com/restaurant/"+place_num+"/review/"
    review_crawling(url)

driver.quit()

0 https://map.naver.com/v5/search/%EC%9A%A9%EC%82%B0%EA%B5%AC%20%EA%B1%B0%EB%A6%AC/place/36200471?c=13185188.0434686,4509248.2664129,5,0,0,0,dh&placePath=%3Fentry%253Dbmp 10
토토가요
1 https://map.naver.com/v5/search/%EC%9A%A9%EC%82%B0%EA%B5%AC%20%EA%B1%B0%EB%A6%AC/place/21714796?c=13175810.1110781,4514948.7228362,5,0,0,0,dh&placePath=%3Fentry%253Dbmp 3
용강동상점가
2 https://map.naver.com/v5/search/%EC%9A%A9%EC%82%B0%EA%B5%AC%20%EA%B1%B0%EB%A6%AC/place/1974980352?c=13185425.7217129,4508773.2584802,5,0,0,0,dh&placePath=%3Fentry%253Dbmp 